In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import re
import pandas as pd
import time
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
import OpenDartReader
import os
import seaborn as sns

In [ ]:
api_key = '31718c7bf232574ee78e6f3f81c922043baad322'
dart = OpenDartReader(api_key)

set_matplotlib_formats('retina')
pd.options.display.max_rows = 50
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus = False)

pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_columns = 60
pd.options.display.max_rows = 200

headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'}
options = webdriver.ChromeOptions()
options.headless = True
options.add_argument('window-size=1920x1080')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

In [ ]:
corp_code = {'삼성생명':'032830',
            '한화생명':'088350'}

dart_list = dart.list(corp_code['한화생명'], start='1999-01-01', kind='A')

report_type_dict = {
    '사업보고서':'A001',
    '반기보고서':'A002',
    '분기보고서':'A003',
}

data_size = 3 # 몇 기간의 데이터를 가져올 건지 결정
report_type_selected = list(report_type_dict.keys())[1] # 1 : 반기보고서

dart_list = dart_list[['corp_name','report_nm','rcept_no']]
dart_list_selected = dart_list[dart_list['report_nm'].str.contains(report_type_selected)].copy()

In [ ]:
corp_nms = list(dart_list_selected['corp_name'])
corp_nm = corp_nms[0] # 회사 이름 

report_nms = list(dart_list_selected['report_nm'])
report_nm = report_nms[0] # 리포트 이름 : 반기보고서 (2020.09)

search_df_nms = ['포괄손익계산서','재무상태표','현금흐름표','자본변동표']
search_df_nm = f'{search_df_nms[0]} {corp_nm}' # 분석하려는 보고서 유형

rcept_nos = list(dart_list_selected['rcept_no'])

In [73]:
data_list = os.listdir()
corp_nm = '한화생명'
match_list = list()
search_df_nm ='포괄손익계산서 한화생명'
for df_nm in data_list:
    if search_df_nm in df_nm:
        match_list.append(df_nm)
        
match_list    

['연결 포괄손익계산서 한화생명 반기보고서 (2018.06).csv',
 '연결 포괄손익계산서 한화생명 반기보고서 (2019.06).csv',
 '연결 포괄손익계산서 한화생명 반기보고서 (2020.06).csv']

In [99]:
df = pd.read_csv('./연결 포괄손익계산서 한화생명 반기보고서 (2020.06).csv')

In [100]:
df = df.drop(0)
df = df.iloc[:,1:5].drop('주 석', axis=1)

In [101]:
df = df.iloc[:,::2]
df = df[df['과 목'].notnull()]

In [102]:
df['과 목'] = df['과 목'].str.strip()
df['과 목'] = df['과 목'].str.split('.').str[-1]
df['과 목'] = df['과 목'].str.split('(').str[0].copy()
df.rename(columns = {'과 목' : '계정과목'}, inplace = True)

In [103]:
df

,계정과목,제 72(당) 반기.1
1,영업수익,13680290409294
2,보험료수익,8102274397804
3,재보험수익,714597132529
4,이자수익,1491854491203
5,구상이익,481094279
6,당기손익인식유가증권 평가 및 처분이익,70889230593
7,매도가능유가증권 평가 및 처분이익,649343985023
8,외환거래이익,1371992452279
9,파생상품평가 및 거래이익,296218996973
10,기타수익,377465498448


In [35]:
df_list = list()

if corp_nm == '삼성생명':
    for i, df_nm in enumerate(match_list):
        df = pd.read_csv('./{}'.format(df_nm), encoding='utf-8-sig')
        df = df.iloc[:,1:3]
        df = df[df['과 목'].notnull()]
        df['과 목'] = df['과 목'].str.strip()
        df['과 목'] = df['과 목'].str.split('.').str[-1]
        df['과 목'] = df['과 목'].str.split('(').str[0].copy()
        df.rename(columns = {'과 목' : '계정과목'}, inplace = True)
        df_list.append(df)
else:
    for i, df_nm in enumerate(match_list):
        df = pd.read_csv('./{}'.format(df_nm), encoding='utf-8-sig')
        df = df.iloc[:,1:5].drop('주 석', axis=1) # 타회사는 여기 부분만 수정!
        df = df[df['과 목'].notnull()]
        df['과 목'] = df['과 목'].str.strip()
        df['과 목'] = df['과 목'].str.split('.').str[-1]
        df['과 목'] = df['과 목'].str.split('(').str[0].copy()
        df.rename(columns = {'과 목' : '계정과목'}, inplace = True)
        df_list.append(df)

In [30]:
df_list_index = list()

for i, val in enumerate(df_list):
    df_index = df_list[i].set_index('계정과목').copy()
    df_list_index.append(df_index)

In [32]:
df_list_index[0] = df_list_index[0].iloc[:,0:1]

,제 70(당) 반기
계정과목,
과 목,3 개 월
영업수익,6588800442117
보험료수익,3729175454486
재보험수익,308938875435
이자수익,761024271901
구상이익,460090842
당기손익인식유가증권 평가 및 처분이익,49999212305
매도가능유가증권 평가 및 처분이익,67909776803
외환거래 및 환산이익,1108611433511
